In [56]:
import boto3
import sagemaker
import pandas as pd
import seaborn as sns
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect
import matplotlib.pyplot as plt

In [57]:
s3_client = boto3.client('s3', region_name='us-east-1')
bucket_name = 'team1ads508finalproject'

In [58]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

s3_public = "s3://team1ads508finalproject"

s3_private = "s3://{}/ADS508_project".format(bucket)

!aws s3 cp --recursive $s3_public/ $s3_private/

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
copy: s3://team1ads508finalproject/bureau.csv to s3://sagemaker-us-east-1-654654578488/ADS508_project/bureau.csv
copy: s3://team1ads508finalproject/application_train.csv to s3://sagemaker-us-east-1-654654578488/ADS508_project/application_train.csv
copy: s3://team1ads508finalproject/previous_application.csv to s3://sagemaker-us-east-1-654654578488/ADS508_project/previous_application.csv


In [59]:
print(s3_private)

s3://sagemaker-us-east-1-654654578488/ADS508_project


In [60]:
%store

Stored variables and their in-db values:


In [61]:
database_name = "dsoaws"

In [62]:
#Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [63]:
conn = connect(region_name='us-east-1', s3_staging_dir=s3_staging_dir)

In [64]:
# Define the table schema
schemas = {
    'bureau': """
        CREATE EXTERNAL TABLE IF NOT EXISTS bureau (
            SK_ID_CURR INT,
            SK_ID_BUREAU INT,
            CREDIT_ACTIVE STRING,
            CREDIT_CURRENCY STRING,
            DAYS_CREDIT INT,
            CREDIT_DAY_OVERDUE INT,
            DAYS_CREDIT_ENDDATE INT,
            DAYS_ENDDATE_FACT INT,
            AMT_CREDIT_MAX_OVERDUE FLOAT,
            CNT_CREDIT_PROLONG INT,
            AMT_CREDIT_SUM FLOAT,
            AMT_CREDIT_SUM_DEBT FLOAT,
            AMT_CREDIT_SUM_LIMIT FLOAT,
            AMT_CREDIT_SUM_OVERDUE FLOAT,
            CREDIT_TYPE STRING,
            DAYS_CREDIT_UPDATE INT,
            AMT_ANNUITY FLOAT
        )
        ROW FORMAT DELIMITED
        FIELDS TERMINATED BY ','
        LOCATION 's3://team1ads508finalproject/'
    """,
    'previous_application': """
        CREATE EXTERNAL TABLE IF NOT EXISTS previous_application (
            SK_ID_PREV INT,
            SK_ID_CURR INT,
            NAME_CONTRACT_TYPE STRING,
            AMT_ANNUITY FLOAT,
            AMT_APPLICATION FLOAT,
            AMT_CREDIT FLOAT,
            AMT_DOWN_PAYMENT FLOAT,
            AMT_GOODS_PRICE FLOAT,
            WEEKDAY_APPR_PROCESS_START STRING,
            HOUR_APPR_PROCESS_START INT,
            FLAG_LAST_APPL_PER_CONTRACT STRING,
            NFLAG_LAST_APPL_IN_DAY INT,
            RATE_DOWN_PAYMENT FLOAT,
            RATE_INTEREST_PRIMARY FLOAT,
            RATE_INTEREST_PRIVILEGED FLOAT,
            NAME_CASH_LOAN_PURPOSE STRING,
            NAME_CONTRACT_STATUS STRING,
            DAYS_DECISION INT,
            NAME_PAYMENT_TYPE STRING,
            CODE_REJECT_REASON STRING,
            NAME_TYPE_SUITE STRING,
            NAME_CLIENT_TYPE STRING,
            NAME_GOODS_CATEGORY STRING,
            NAME_PORTFOLIO STRING,
            NAME_PRODUCT_TYPE STRING,
            CHANNEL_TYPE STRING,
            SELLERPLACE_AREA INT,
            NAME_SELLER_INDUSTRY STRING,
            CNT_PAYMENT INT,
            NAME_YIELD_GROUP STRING,
            PRODUCT_COMBINATION STRING,
            DAYS_FIRST_DRAWING INT,
            DAYS_FIRST_DUE INT,
            DAYS_LAST_DUE_1ST_VERSION INT,
            DAYS_LAST_DUE INT,
            DAYS_TERMINATION INT,
            NFLAG_INSURED_ON_APPROVAL INT
        )
        ROW FORMAT DELIMITED
        FIELDS TERMINATED BY ','
        LOCATION 's3://team1ads508finalproject/'
    """,
    'application': """
        CREATE EXTERNAL TABLE IF NOT EXISTS application (
            SK_ID_CURR INT,
            TARGET INT,
            NAME_CONTRACT_TYPE STRING,
            CODE_GENDER STRING,
            FLAG_OWN_CAR STRING,
            FLAG_OWN_REALTY STRING,
            CNT_CHILDREN INT,
            AMT_INCOME_TOTAL FLOAT,
            AMT_CREDIT FLOAT,
            AMT_ANNUITY FLOAT,
            AMT_GOODS_PRICE FLOAT,
            NAME_TYPE_SUITE STRING,
            NAME_INCOME_TYPE STRING,
            NAME_EDUCATION_TYPE STRING,
            NAME_FAMILY_STATUS STRING,
            NAME_HOUSING_TYPE STRING,
            REGION_POPULATION_RELATIVE FLOAT,
            DAYS_BIRTH INT,
            DAYS_EMPLOYED INT,
            DAYS_REGISTRATION INT,
            DAYS_ID_PUBLISH INT,
            OWN_CAR_AGE INT,
            FLAG_MOBIL INT,
            FLAG_EMP_PHONE INT,
            FLAG_WORK_PHONE INT,
            FLAG_CONT_MOBILE INT,
            FLAG_PHONE INT,
            FLAG_EMAIL INT,
            OCCUPATION_TYPE STRING,
            CNT_FAM_MEMBERS FLOAT,
            REGION_RATING_CLIENT INT,
            REGION_RATING_CLIENT_W_CITY INT,
            WEEKDAY_APPR_PROCESS_START STRING,
            HOUR_APPR_PROCESS_START INT,
            REG_REGION_NOT_LIVE_REGION INT,
            REG_REGION_NOT_WORK_REGION INT,
            LIVE_REGION_NOT_WORK_REGION INT,
            REG_CITY_NOT_LIVE_CITY INT,
            REG_CITY_NOT_WORK_CITY INT,
            LIVE_CITY_NOT_WORK_CITY INT,
            ORGANIZATION_TYPE STRING,
            EXT_SOURCE_1 FLOAT,
            EXT_SOURCE_2 FLOAT,
            EXT_SOURCE_3 FLOAT,
            APARTMENTS_AVG FLOAT,
            BASEMENTAREA_AVG FLOAT,
            YEARS_BEGINEXPLUATATION_AVG FLOAT,
            YEARS_BUILD_AVG FLOAT,
            COMMONAREA_AVG FLOAT,
            ELEVATORS_AVG FLOAT,
            ENTRANCES_AVG FLOAT,
            FLOORSMAX_AVG FLOAT,
            FLOORSMIN_AVG FLOAT,
            LANDAREA_AVG FLOAT,
            LIVINGAPARTMENTS_AVG FLOAT,
            LIVINGAREA_AVG FLOAT,
            NONLIVINGAPARTMENTS_AVG FLOAT,
            NONLIVINGAREA_AVG FLOAT,
            APARTMENTS_MODE FLOAT,
            BASEMENTAREA_MODE FLOAT,
            YEARS_BEGINEXPLUATATION_MODE FLOAT,
            YEARS_BUILD_MODE FLOAT,
            COMMONAREA_MODE FLOAT,
            ELEVATORS_MODE FLOAT,
            ENTRANCES_MODE FLOAT,
            FLOORSMAX_MODE FLOAT,
            FLOORSMIN_MODE FLOAT,
            LANDAREA_MODE FLOAT,
            LIVINGAPARTMENTS_MODE FLOAT,
            LIVINGAREA_MODE FLOAT,
            NONLIVINGAPARTMENTS_MODE FLOAT,
            NONLIVINGAREA_MODE FLOAT,
            APARTMENTS_MEDI FLOAT,
            BASEMENTAREA_MEDI FLOAT,
            YEARS_BEGINEXPLUATATION_MEDI FLOAT,
            YEARS_BUILD_MEDI FLOAT,
            COMMONAREA_MEDI FLOAT,
            ELEVATORS_MEDI FLOAT,
            ENTRANCES_MEDI FLOAT,
            FLOORSMAX_MEDI FLOAT,
            FLOORSMIN_MEDI FLOAT,
            LANDAREA_MEDI FLOAT,
            LIVINGAPARTMENTS_MEDI FLOAT,
            LIVINGAREA_MEDI FLOAT,
            NONLIVINGAPARTMENTS_MEDI FLOAT,
            NONLIVINGAREA_MEDI FLOAT,
            FONDKAPREMONT_MODE STRING,
            HOUSETYPE_MODE STRING,
            TOTALAREA_MODE FLOAT,
            WALLSMATERIAL_MODE STRING,
            EMERGENCYSTATE_MODE STRING,
            OBS_30_CNT_SOCIAL_CIRCLE FLOAT,
            DEF_30_CNT_SOCIAL_CIRCLE FLOAT,
            OBS_60_CNT_SOCIAL_CIRCLE FLOAT,
            DEF_60_CNT_SOCIAL_CIRCLE FLOAT,
            DAYS_LAST_PHONE_CHANGE INT,
            FLAG_DOCUMENT_2 INT,
            FLAG_DOCUMENT_3 INT,
            FLAG_DOCUMENT_4 INT,
            FLAG_DOCUMENT_5 INT,
            FLAG_DOCUMENT_6 INT,
            FLAG_DOCUMENT_7 INT,
            FLAG_DOCUMENT_8 INT,
            FLAG_DOCUMENT_9 INT,
            FLAG_DOCUMENT_10 INT,
            FLAG_DOCUMENT_11 INT,
            FLAG_DOCUMENT_12 INT,
            FLAG_DOCUMENT_13 INT,
            FLAG_DOCUMENT_14 INT,
            FLAG_DOCUMENT_15 INT,
            FLAG_DOCUMENT_16 INT,
            FLAG_DOCUMENT_17 INT,
            FLAG_DOCUMENT_18 INT,
            FLAG_DOCUMENT_19 INT,
            FLAG_DOCUMENT_20 INT,
            FLAG_DOCUMENT_21 INT,
            AMT_REQ_CREDIT_BUREAU_HOUR FLOAT,
            AMT_REQ_CREDIT_BUREAU_DAY FLOAT,
            AMT_REQ_CREDIT_BUREAU_WEEK FLOAT,
            AMT_REQ_CREDIT_BUREAU_MON FLOAT,
            AMT_REQ_CREDIT_BUREAU_QRT FLOAT,
            AMT_REQ_CREDIT_BUREAU_YEAR FLOAT
        )
        ROW FORMAT DELIMITED
        FIELDS TERMINATED BY ','
        LOCATION 's3://team1ads508finalproject/'
    """
}

# Create tables in Athena
for table_name, schema in schemas.items():
    print(f"Creating table: Credit")
    cursor = conn.cursor()
    cursor.execute(schema)

# Show tables 
query = "SHOW TABLES"
df_show = pd.read_sql(query, conn)
df_show.head(5)

Creating table: Credit
Creating table: Credit
Creating table: Credit


/tmp/ipykernel_4356/2506220880.py:210: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(query, conn)


,tab_name
0,application
1,bureau
2,previous_application


In [65]:
# Do EDA on each dataset then identify which columns are important and then change join query below!
join_query = """
SELECT *
FROM application a
INNER JOIN bureau b ON a.SK_ID_CURR = b.SK_ID_CURR
INNER JOIN previous_application p ON a.SK_ID_CURR = p.SK_ID_CURR
LIMIT 30000;
"""

# Convert to Dataframe
combined_df = pd.read_sql(join_query, conn)

# Display the result
print(combined_df)

/tmp/ipykernel_4356/84662886.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  combined_df = pd.read_sql(join_query, conn)


       sk_id_curr  target name_contract_type code_gender flag_own_car  \
0          254371       0         Cash loans           F            Y   
1          254371       0         Cash loans           F            Y   
2          254371       0         Cash loans           F            Y   
3          254371       0         Cash loans           F            Y   
4          254371       0         Cash loans           F            Y   
...           ...     ...                ...         ...          ...   
29995      182614       0         Cash loans           M            N   
29996      182614       0         Cash loans           M            N   
29997      182614       0         Cash loans           M            N   
29998      182614       0         Cash loans           M            N   
29999      182614       0         Cash loans           M            N   

      flag_own_realty  cnt_children  amt_income_total  amt_credit  \
0                   Y             0          216000.0 

In [66]:
# Unique Values
unique = combined_df.nunique()
unique

sk_id_curr                   1010
target                          2
name_contract_type              2
code_gender                     2
flag_own_car                    2
                             ... 
days_first_due               1615
days_last_due_1st_version    1786
days_last_due                1496
days_termination             1428
nflag_insured_on_approval     134
Length: 176, dtype: int64

In [67]:
# Null Values
null_values = combined_df.isnull().sum()
null_values

sk_id_curr                       0
target                           0
name_contract_type               0
code_gender                      0
flag_own_car                     0
                             ...  
days_first_due               11956
days_last_due_1st_version    11956
days_last_due                11956
days_termination             11956
nflag_insured_on_approval    11956
Length: 176, dtype: int64

In [68]:
# Drop columns with null values
final_df = combined_df.dropna(axis=1)

# Display Clean Dataframe
print(final_df)


       sk_id_curr  target name_contract_type code_gender flag_own_car  \
0          254371       0         Cash loans           F            Y   
1          254371       0         Cash loans           F            Y   
2          254371       0         Cash loans           F            Y   
3          254371       0         Cash loans           F            Y   
4          254371       0         Cash loans           F            Y   
...           ...     ...                ...         ...          ...   
29995      182614       0         Cash loans           M            N   
29996      182614       0         Cash loans           M            N   
29997      182614       0         Cash loans           M            N   
29998      182614       0         Cash loans           M            N   
29999      182614       0         Cash loans           M            N   

      flag_own_realty  cnt_children  amt_income_total  amt_credit  \
0                   Y             0          216000.0 

In [69]:
final_df.describe()

,sk_id_curr,target,cnt_children,amt_income_total,amt_credit,amt_annuity,days_birth,days_employed,days_registration,days_id_publish,...,sk_id_curr,sk_id_bureau,cnt_credit_prolong,sk_id_prev,sk_id_curr,amt_application,amt_credit,hour_appr_process_start,nflag_last_appl_in_day,days_decision
count,30000.000000,30000.000000,30000.000000,3.000000e+04,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,3.000000e+04,30000.000000,3.000000e+04,30000.000000,3.000000e+04,3.000000e+04,30000.000000,30000.000000,30000.000000
mean,209859.896500,0.091333,0.404967,1.827616e+05,5.967166e+05,27490.06845,-16011.440867,78875.746633,-2530.436900,-3332.285667,...,209859.896500,4.987885e+06,4226.712967,1.923657e+06,209859.896500,1.788806e+05,2.003789e+05,12.379633,0.994033,-928.859167
std,47568.264964,0.288087,0.707049,9.206301e+04,3.936774e+05,13835.47240,5609.472527,153609.126850,30099.340739,1609.934247,...,47568.264964,2.201757e+06,11238.504034,5.272977e+05,47568.264964,3.016916e+05,3.266105e+05,3.379471,0.077015,801.943967
min,172024.000000,0.000000,0.000000,2.700000e+04,5.212800e+04,4450.50000,-25201.000000,-23324.000000,-17736.000000,-13273.000000,...,172024.000000,0.000000e+00,0.000000,1.000336e+06,172024.000000,0.000000e+00,0.000000e+00,2.000000,0.000000,-2917.000000
25%,172614.000000,0.000000,0.000000,1.125000e+05,2.814930e+05,16830.00000,-20258.000000,-2888.000000,-7134.000000,-4449.000000,...,172614.000000,5.211315e+06,0.000000,1.473770e+06,172614.000000,2.174998e+04,2.657250e+04,10.000000,1.000000,-1448.000000
50%,182671.000000,0.000000,0.000000,1.575000e+05,4.995000e+05,25407.00000,-16308.000000,-1544.000000,-4385.000000,-3601.000000,...,182671.000000,5.739591e+06,0.000000,1.920767e+06,182671.000000,7.006050e+04,7.742250e+04,12.000000,1.000000,-636.000000
75%,244760.000000,0.000000,1.000000,2.250000e+05,7.908300e+05,34596.00000,-12722.500000,-241.000000,-2418.000000,-2307.000000,...,244760.000000,6.285726e+06,0.000000,2.388524e+06,244760.000000,1.800000e+05,2.206620e+05,15.000000,1.000000,-283.000000
max,327096.000000,1.000000,6.000000,2.070000e+06,2.250000e+06,109597.50000,0.000000,365243.000000,365243.000000,-4.000000,...,327096.000000,6.843347e+06,109597.000000,2.844461e+06,327096.000000,2.700000e+06,3.020760e+06,23.000000,1.000000,-2.000000


In [70]:
print(final_df.columns)

Index(['sk_id_curr', 'target', 'name_contract_type', 'code_gender',
       'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total',
       'amt_credit', 'amt_annuity', 'name_type_suite', 'name_income_type',
       'name_education_type', 'name_family_status', 'name_housing_type',
       'days_birth', 'days_employed', 'days_registration', 'days_id_publish',
       'flag_emp_phone', 'flag_work_phone', 'flag_cont_mobile', 'flag_phone',
       'flag_email', 'occupation_type', 'region_rating_client',
       'region_rating_client_w_city', 'weekday_appr_process_start',
       'reg_region_not_live_region', 'reg_region_not_work_region',
       'live_region_not_work_region', 'reg_city_not_live_city',
       'reg_city_not_work_city', 'live_city_not_work_city',
       'organization_type', 'fondkapremont_mode', 'housetype_mode',
       'wallsmaterial_mode', 'emergencystate_mode', 'flag_document_2',
       'flag_document_3', 'flag_document_4', 'flag_document_5',
       'flag_document_6'

In [71]:
final_df = final_df[['sk_id_curr', 'target', 'name_contract_type', 'code_gender',
       'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total',
       'amt_credit', 'amt_annuity', 'credit_active', 'amt_income_total',
       'amt_credit', 'amt_annuity', 'name_type_suite', 'name_income_type',
       'name_education_type', 'name_family_status', 'name_housing_type', 'weekday_appr_process_start', 'organization_type', 'name_cash_loan_purpose', 'name_contract_status', 'days_decision',
       'name_payment_type', 'code_reject_reason', 'name_type_suite', 'name_client_type', 'name_portfolio']]
final_df

,sk_id_curr,sk_id_curr,sk_id_curr,target,name_contract_type,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,...,organization_type,name_cash_loan_purpose,name_contract_status,days_decision,name_payment_type,code_reject_reason,name_type_suite,name_type_suite,name_client_type,name_portfolio
0,254371,254371,254371,0,Cash loans,Consumer loans,F,Y,Y,0,...,Other,XAP,Approved,-1456,Cash through the bank,XAP,Unaccompanied,Unaccompanied,Refreshed,POS
1,254371,254371,254371,0,Cash loans,Cash loans,F,Y,Y,0,...,Other,XNA,Approved,-136,XNA,XAP,Unaccompanied,Unaccompanied,Repeater,Cash
2,254371,254371,254371,0,Cash loans,Revolving loans,F,Y,Y,0,...,Other,XAP,Approved,-1276,XNA,XAP,Unaccompanied,,Repeater,Cards
3,254371,254371,254371,0,Cash loans,Consumer loans,F,Y,Y,0,...,Other,XAP,Approved,-526,Cash through the bank,XAP,Unaccompanied,,Repeater,POS
4,254371,254371,254371,0,Cash loans,Consumer loans,F,Y,Y,0,...,Other,XAP,Approved,-1456,Cash through the bank,XAP,Unaccompanied,Unaccompanied,Refreshed,POS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,182614,182614,182614,0,Cash loans,Cash loans,M,N,Y,0,...,XNA,XNA,Canceled,-348,XNA,XAP,Unaccompanied,,Repeater,XNA
29996,182614,182614,182614,0,Cash loans,Cash loans,M,N,Y,0,...,XNA,XNA,Approved,-348,Cash through the bank,XAP,Unaccompanied,,Repeater,Cash
29997,182614,182614,182614,0,Cash loans,Revolving loans,M,N,Y,0,...,XNA,XAP,Refused,-959,XNA,HC,Unaccompanied,,Repeater,Cards
29998,182614,182614,182614,0,Cash loans,Cash loans,M,N,Y,0,...,XNA,Repairs,Refused,-695,Cash through the bank,LIMIT,Unaccompanied,Unaccompanied,Repeater,Cash


In [72]:
duplicates = final_df.duplicated(subset=['sk_id_curr'])
print("Duplicate rows:", duplicates.sum())

final_df.drop_duplicates(subset=['sk_id_curr'], inplace=True)
print(final_df)

Duplicate rows: 28990
       sk_id_curr  sk_id_curr  sk_id_curr  target name_contract_type  \
0          254371      254371      254371       0         Cash loans   
24         254405      254405      254405       0         Cash loans   
25         254418      254418      254418       0         Cash loans   
31         254429      254429      254429       0         Cash loans   
37         254431      254431      254431       0         Cash loans   
...           ...         ...         ...     ...                ...   
29828      182547      182547      182547       0         Cash loans   
29880      182557      182557      182557       0         Cash loans   
29940      182587      182587      182587       0         Cash loans   
29949      182591      182591      182591       0         Cash loans   
29963      182614      182614      182614       0         Cash loans   

      name_contract_type code_gender flag_own_car flag_own_realty  \
0         Consumer loans           F        

In [ ]:
# Create a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(final_df, annot=True, cmap='viridis', fmt='d')
plt.title('Feature Assessment Heatmap')
plt.show()